In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import torch
import random

In [2]:
SEARCH_SPACE = {
    'k_size_a': [1, 3, 5, 7],
    'k_size_b': [1, 3, 5, 7],
    'out_channels_a': [8, 16, 32, 64],
    'out_channels_b': [8, 16, 32, 64],
    'include_pool_a': [True, False],
    'include_pool_b': [True, False],
    'pool_type_a': ['max_pooling','avg_pooling'],
    'pool_type_b': ['max_pooling','avg_pooling'],
    'activation_type_a': ['relu', 'tanh', 'elu', 'selu'],
    'activation_type_b': ['relu', 'tanh', 'elu', 'selu'], 
    'include_b': [True, False],
    'include_BN_a': [True, False],
    'include_BN_b': [True, False],
    'skip_connection': [True, False],
}

In [3]:
class Chromosome:
    def __init__(self,phase:int,prev_best,genes:dict):
        self.phase = phase
        self.prev_best = prev_best
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.genes = genes
        self.out_dimensions = prev_best.out_dimensions if phase!=0 else 32
        self.model:nn.Module = self.build_model()
        # self.fitness = self.fitness_function()

    def build_model(self)->nn.Module:
        if(self.prev_best!=None):
            prev_best_model:nn.Module = self.prev_best.model
        new_model_modules = []
        if(self.phase!=0):
            layer_a = nn.Conv2d(self.prev_best.genes['out_channels_b'] if self.prev_best.genes['include_b'] else self.prev_best.genes['out_channels_a'],self.genes['out_channels_a'],self.genes['k_size_a'])
        else:
            layer_a = nn.Conv2d(3,self.genes['out_channels_a'],self.genes['k_size_a'])
        self.out_dimensions = (self.out_dimensions-self.genes['k_size_a']+1)
        new_model_modules.append(layer_a)
        if(self.genes['activation_type_a']=='relu'):
            new_model_modules.append(nn.ReLU())
        else:
            new_model_modules.append(nn.Tanh())
        if(self.genes['include_pool_a']):
            if(self.genes['pool_type_a']=='max_pooling'):
                new_model_modules.append(nn.MaxPool2d(2,2))
                self.out_dimensions = self.out_dimensions//2
            elif(self.genes['pool_type_a']=='avg_pooling'):
                new_model_modules.append(nn.AvgPool2d(2,2))
                self.out_dimensions = self.out_dimensions//2
            else:
                raise Exception('Invalid pool type (a layer)')
        
        if(self.genes['include_BN_a']):
            new_model_modules.append(nn.BatchNorm2d(self.genes['out_channels_a']))
        
        if(self.genes['include_b'] or self.phase==0):
            layer_b = nn.Conv2d(self.genes['out_channels_a'],self.genes['out_channels_b'],self.genes['k_size_b'])
            self.out_dimensions = (self.out_dimensions-self.genes['k_size_b']+1)
            new_model_modules.append(layer_b)
            if(self.genes['activation_type_b']=='relu'):
                new_model_modules.append(nn.ReLU())
            else:
                new_model_modules.append(nn.Tanh())
            
            if(self.genes['include_pool_b']):
                if(self.genes['pool_type_b']=='max_pooling'):
                    new_model_modules.append(nn.MaxPool2d(2,2))
                    self.out_dimensions = self.out_dimensions//2
                elif(self.genes['pool_type_b']=='avg_pooling'):
                    new_model_modules.append(nn.AvgPool2d(2,2))
                    self.out_dimensions = self.out_dimensions//2
                else:
                    raise Exception('Invalid pool type (b layer)')
                
            if(self.genes['include_BN_b']):
                new_model_modules.append(nn.BatchNorm2d(self.genes['out_channels_b']))
        if(self.phase!=0):
            new_model = nn.Sequential(prev_best_model,*new_model_modules)
        else:
            new_model = nn.Sequential(*new_model_modules)
        return new_model            

    def fitness_function(self,train_loader,test_loader)->float:
        self.model = self.model.to(self.device)
        num_epochs = 5
        criterion = F.nll_loss
        optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        for epoch in range(num_epochs):
            pbar = tqdm(train_loader)      
            self.model.train()
            for batch_idx, (data, target) in enumerate(pbar):
                data, target = data.to(self.device), target.to(self.device)
                optimizer.zero_grad()
                output = self.forward(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()
                pbar.set_description(desc= f'epoch: {epoch} loss={loss.item()} batch_id={batch_idx}')

        

        
    def forward(self,x:torch.Tensor):
        if(x.shape[0]==1):
            return self._forward_one_example(x)
        else:
            outputs = torch.zeros(x.shape[0],10,device = self.device)
            for i in range(x.shape[0]):
                outputs[i] = self._forward_one_example(x[i].unsqueeze(0))
            return outputs

    def _forward_one_example(self,x:torch.Tensor):
        x = self.model(x)
        if(self.genes['skip_connection']):
            if(self.phase==0):
                y = nn.Conv2d(3,self.genes['out_channels_b'],1)(x)
            else:
                y = nn.Conv2d(self.prev_best.genes['out_channels_b'] if self.prev_best.genes['include_b'] else self.prev_best.genes['out_channels_a'],self.genes['out_channels_b'] if self.genes['include_b'] else self.genes['out_channels_a'],1)(x)
            x = x+y
        x = x.flatten()
        x = nn.Linear(x.shape[0],10,device=self.device)(x)
        return F.log_softmax(x,dim = 0)

    def crossover(self, chromosome):
        genes1 = self.genes
        genes2 = chromosome.genes
        keys = genes1.keys()
        new_genes = {}
        for key in keys:
            new_genes[key] = random.choice([genes1[key], genes2[key]])
        new_chromosome = Chromosome(self.phase, self.prev_best, new_genes)
        return new_chromosome 
    
    def mutation(self):
        mutated_gene = random.choice(list(self.genes.keys()))
        possible_values = [value for value in SEARCH_SPACE[mutated_gene]]
        possible_values.remove(self.genes[mutated_gene])
        new_gene_value = random.choice(possible_values)
        self.genes[mutated_gene] = new_gene_value
        

        

In [4]:
random_genes1 = {
    'k_size_a': 3,
    'k_size_b': 3,
    'out_channels_a': 16,
    'out_channels_b': 16,
    'include_pool_a': True,
    'include_pool_b': False,
    'pool_type_a': 'max_pooling',
    'pool_type_b': 'max_pooling',
    'activation_type_a': 'relu',
    'activation_type_b': 'relu',
    'include_b': False,
    'include_BN_a': True,
    'include_BN_b': True,
    'skip_connection': False,
}
random_genes2 = {
    'k_size_a': 5,
    'k_size_b': 3,
    'out_channels_a': 32,
    'out_channels_b': 64,
    'include_pool_a': False,
    'include_pool_b': True,
    'pool_type_a': 'avg_pooling',
    'pool_type_b': 'max_pooling',
    'activation_type_a': 'elu',
    'activation_type_b': 'tanh',
    'include_b': True,
    'include_BN_a': False,
    'include_BN_b': True,
    'skip_connection': True,
}

In [5]:
def crossover(genes1, genes2):
    keys = genes1.keys()
    new_genes = {}
    for key in keys:
        new_genes[key] = random.choice([genes1[key], genes2[key]])
    return new_genes

In [6]:
crossover(random_genes1, random_genes2)

{'k_size_a': 3,
 'k_size_b': 3,
 'out_channels_a': 32,
 'out_channels_b': 16,
 'include_pool_a': True,
 'include_pool_b': True,
 'pool_type_a': 'max_pooling',
 'pool_type_b': 'max_pooling',
 'activation_type_a': 'relu',
 'activation_type_b': 'tanh',
 'include_b': True,
 'include_BN_a': True,
 'include_BN_b': True,
 'skip_connection': True}

In [7]:
def mutation(genes):
    mutated_gene = random.choice(list(genes.keys()))
    print(mutated_gene)
    possible_values = [value for value in SEARCH_SPACE[mutated_gene]]
    possible_values.remove(genes[mutated_gene])
    new_gene_value = random.choice(possible_values)
    genes[mutated_gene] = new_gene_value
    print(genes)

In [8]:
mutation(random_genes2)

k_size_b
{'k_size_a': 5, 'k_size_b': 7, 'out_channels_a': 32, 'out_channels_b': 64, 'include_pool_a': False, 'include_pool_b': True, 'pool_type_a': 'avg_pooling', 'pool_type_b': 'max_pooling', 'activation_type_a': 'elu', 'activation_type_b': 'tanh', 'include_b': True, 'include_BN_a': False, 'include_BN_b': True, 'skip_connection': True}


In [9]:
d = {'a': 1, 'b': 3}
d.keys()

dict_keys(['a', 'b'])